In [1]:
#source: https://www.tensorflow.org/guide/keras/sequential_model
#intro to keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

the sequential model is appropriate for **a plain stack of layrs** where each layer has **exactly one input tensor and out output tensor**.

the following sequential model:

In [3]:
model = keras.Sequential(
                            [
                            layers.Dense(2, activation="relu", name="layer1"),
                            layers.Dense(3, activation="relu", name="layer2"),
                            layers.Dense(4, name="layer3")
                            ]
)

#call model on test input
x = tf.ones((3, 3))
y = model(x)


is equivalent to this function:

In [8]:
#create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

y_2 = layer3(layer2(layer1(x)))

**creating a Sequential model:**

you can creat a sequential model by passing a list of alayers to the sequential constructor (above in the second code cell)

the layers are accessable via the layers attribute

In [9]:
print(model.layers)

[<tensorflow.python.keras.layers.core.Dense object at 0x00000250CC209AF0>, <tensorflow.python.keras.layers.core.Dense object at 0x00000250CC209760>, <tensorflow.python.keras.layers.core.Dense object at 0x00000250CC3DDDC0>]


you can also add to the model, or create a model incrementally via the **add()** method

In [10]:
model2 = keras.Sequential()

model2.add(layers.Dense(2, activation="relu"))
model2.add(layers.Dense(3, activation="relu"))
model2.add(layers.Dense(4))

you can remove layers via the **pop()** function. this makes the keras model behave like a stack

In [11]:
print(len(model2.layers))
model2.pop()
print(len(model2.layers))

3
2


a sequential constructor also accepts a name argument, just like any layer. this is useful to annotate TensorBoard graphs with meaningful names

In [13]:
model3 = keras.Sequential(name="my_sequential")
model3.add(layers.Dense(2, activation="relu", name="layer1"))
model3.add(layers.Dense(3, activation="relu", name="layer2"))
model3.add(layers.Dense(4, name="layer3"))


all layers need to know the shape of thier input to create their weights so, initially, a layer will have no weights. it creates weights the first time it is called on an input. so the shape of the wieghts depends on the shape of the inputs.

In [22]:
layer = layers.Dense(4)
print(layer.weights) #empty

x = tf.ones((1, 4))
print(x)
y = layer(x)

print(layer.weights)

[]
tf.Tensor([[1. 1. 1. 1.]], shape=(1, 4), dtype=float32)
[<tf.Variable 'dense_10/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.27112478,  0.65860194,  0.23354167,  0.06971657],
       [ 0.2677253 ,  0.57646996, -0.7287724 ,  0.5109716 ],
       [-0.4640098 , -0.7533757 ,  0.04875368,  0.33155388],
       [ 0.43726653,  0.7513469 ,  0.8567032 ,  0.19465286]],
      dtype=float32)>, <tf.Variable 'dense_10/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]


In [24]:
print(y)

tf.Tensor([[-0.03014275  1.2330431   0.41022617  1.106895  ]], shape=(1, 4), dtype=float32)


this also applies to sequential models. when you instantiate a sequential model without an input shape, it isn't "built" just yet. the wieghts are created when the model first sees some input data.

once a model is built then you can call its **sumarry()** method to display the contents

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (3, 2)                    8         
_________________________________________________________________
layer2 (Dense)               (3, 3)                    9         
_________________________________________________________________
layer3 (Dense)               (3, 4)                    16        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


to have the weights (randomly) intialized in advance, you pass an **Input** object to the model so it knows the expected shape of the input

In [27]:
myModel = keras.Sequential(name="another Sequential model")
myModel.add(keras.Input(shape=(4,)))
myModel.add(layers.Dense(2, activation="relu"))

myModel.summary()

Model: "another Sequential model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


alternatively you can just pass an **input_shape** optional argument to your first layer

**a common debuggin workflow: add() + summary()**

when building a new architecture, its useful to incrementally stack layers with add(). you can then monitor and debug the progress of the architecture with the summary() method. for example, it enables you to monitor hwo a stack of Conv2d and MaxPooling2D layers are downsampling image feature maps.

In [28]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
____________________________

once a model is built, you will want to:

train your model, evaluate it, and run inference: https://www.tensorflow.org/guide/keras/train_and_evaluate/
save your model to disk and restore it: https://www.tensorflow.org/guide/keras/save_and_serialize/
speed up model training by using GPU resources: https://keras.io/guides/distributed_training/

**feature extraction with a Sequential Model**
Once a Sequential model has been built, it behaves like a **functional API model** (https://www.tensorflow.org/guide/keras/functional/). meaning that every layer has **input** and **output** attributes.

In [30]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

#Here's a similar example that only extract features from one layer:

initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)
